In [1]:
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import cv2


Read the CSV containing the Image Metadata

In [2]:
styles = pd.read_csv(filepath_or_buffer="C:/Users/weime/Documents/Python Scripts/CircularFashion/Data/fashion-dataset/styles.csv",
            delimiter=",", encoding="utf-8", on_bad_lines = "skip")

Filter the df on Jeans

In [3]:
Shirts = styles[styles["articleType"].str.match('.*[Ss]weaters.*|.*[Ss]hirt.*|.*[Ss]uits.*|.*[Jj]ackets.*')]
Shirts

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012.0,Ethnic,Fabindia Men Striped Green Shirt
7,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012.0,Casual,Jealous 21 Women Purple Shirt
...,...,...,...,...,...,...,...,...,...,...
44412,1644,Men,Apparel,Topwear,Tshirts,Red,Fall,2010.0,Sports,Kipsta Men Loose Fit Round Neck Jersey Red
44414,30614,Men,Apparel,Topwear,Tshirts,Black,Summer,2012.0,Sports,Nike Men Striped Black Jersey
44415,13496,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Chimp Men Teja Main Hoon Blue Tshirts
44417,12544,Women,Apparel,Topwear,Tshirts,Peach,Fall,2011.0,Casual,Tantra Women Printed Peach T-shirt


Group the dataset after distinguishing attributes and sample from each group. As the smallest group is of size 1 and there are around 60 groups anyways we will only sample 1 image from every group. As there are still 700 images left I will subsample 100 of them.

In [4]:

TrainingData = Shirts.groupby(['gender', 'baseColour', 'season', 'year', 'usage'])


TrainingData = TrainingData.apply(lambda x: x.sample(n = 1))
TrainingData = TrainingData.sample(100)

Define Functions that creat the image paths (Original/source and training)

In [5]:
def create_source_path(id):     
    return "C:/Users/weime/Documents/Python Scripts/CircularFashion/Data/fashion-dataset/images/"+ str(id) + ".jpg"
def create_training_path(id, extension):     
    return "C:/Users/weime/Documents/Python Scripts/CircularFashion/Data/fashion-dataset/targets/"+ str(id) + extension + ".jpg"

Define functions that allow the marking of an area. 


In [6]:

# function that allows the marking of an area using OpenCV
def create_target(path, extension):
    global img
    global clone
    
    # read the image
    img = mpimg.imread(path)
    # create a deep copy of the Image in order to be able to reset it
    clone = img.copy()
    
    # start the OpenCV session
    cv2.startWindowThread()
    cv2.namedWindow("image", cv2.WINDOW_NORMAL)
    cv2.setMouseCallback("image", mark_area)
    
    # show the Image
    cv2.imshow("image", img)

    while True:
        # display the image and wait for a keypress
        key = cv2.waitKey(100) & 0xFF
        cv2.imshow("image", img)
        # if the 'r' key is pressed, reset the image
        if key == ord("r"):
            img = clone.copy()
        # if the 'c' key is pressed, break from the loop
        if key == ord("c"):
            break
    
    cv2.destroyAllWindows()

    img1 = np.zeros(shape = img.shape)
    img1[refPt[0][1]:refPt[1][1],refPt[0][0]:refPt[1][0],:] = 1
    mpimg.imsave(arr = img1, fname = path.replace("images", "targets").replace(".jpg", extension + ".jpg"))





def mark_area(event,x,y,flags,param):
    
    #save the coordinates of left mouse down
    if event == cv2.EVENT_LBUTTONDOWN:
        global img
        global clone
        global refPt
        # reset the image, this lead to a refreshed image upon a second selection
        # if this fails press r
        img = clone.copy()
        refPt = [(x, y)]
    
    #save the coordinates of left mouse up and mark the corresponding area    
    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
               
        img[refPt[0][1]:refPt[1][1],refPt[0][0]:refPt[1][0],:] = img[refPt[0][1]:refPt[1][1],refPt[0][0]:refPt[1][0],:] / 2

        


Create a target for all images in the training data by selecting a target area.

In [14]:

for i in range(TrainingData.shape[0]):
    
    extension = "_neckline"
    if(os.path.exists(create_training_path(TrainingData["id"][i], extension) )):
        continue
    
    if(os.path.exists(create_source_path(TrainingData["id"][i]) )):
        create_target(path = create_source_path(TrainingData["id"][i]), extension = extension)
